In [36]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys 

sns.set()
sys.path.append("../src/")

from models import SimulateStatement, Model
from modelstats import ModelStats
from dataclasses import dataclass
from crop import crop_statements_until_t_by_politician, crop_statements_until_t
from datetime import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
df = pd.read_csv("../data/DATA_FINAL.csv")
df = df.sort_values(by=['time'])

In [38]:
df.time = pd.to_datetime(df.time)


In [39]:
tau = crop_statements_until_t_by_politician(df,'2019-07-11 15:30:16', 422)

In [40]:
# sys.path.append("../data/")

data_path = "../data/DATA_FINAL.csv"
data_deputados_path = "../data/DEPUTADOS_FINAL.csv"

Experimento = ModelStats(data_path, data_deputados_path)

lambd = 0.6
delta = 0.3
lag = 100

In [41]:
Experimento.get_rates()

{422: 24.877301311724537,
 307: 7.883637152777778,
 223: 8.23239162457176,
 477: 16.45727272726852,
 346: 12.129429309108797,
 120: 24.061726190474538,
 299: 6.993602053136574,
 430: 2.057789058831019,
 470: 6.969468095810185,
 513: nan,
 34: 2.900762626261574,
 348: 2.452665598287037,
 395: 22.49662533068287,
 195: 10.569191919189816,
 127: 3.3411374605902777,
 280: 2.869059343425926,
 490: 3.6255687446064817,
 9: 37.08677083333333,
 455: 2.639154896412037,
 74: 8.554536146712962,
 26: 7.421552579363426,
 146: 3.9984974477662036,
 144: nan,
 488: 7.002398463796297,
 377: 6.429955632708333,
 405: 30.866793981481482,
 71: 6.162935185185185,
 106: 4.969742010150463,
 459: 69.12391203703703,
 485: 10.920719246030092,
 272: 11.544189814814814,
 342: 3.052205687824074,
 450: 2.7613284464930556,
 176: 8.242672968101852,
 136: 16.689340277777777,
 169: 35.05243344907407,
 381: 8.714719498900463,
 62: 7.082319775127314,
 443: 10.903496260682871,
 95: nan,
 441: 13.887509920625,
 214: 16.438040

In [34]:
politicians_opinion_histograms = Experimento.get_score_histogram(lambd,delta,lag)

100%|██████████| 314/314 [02:37<00:00,  1.99it/s]


In [2]:
from datetime import datetime
from dataclasses import dataclass
from typing import List

@dataclass
class PoliticianOpinion:
    politician_name: str
    opinion_score: float

@dataclass
class PoliticiansOpinionInTime:
    politician_opinions: List[PoliticianOpinion]
    datetime: datetime

def get_politician_trajectories(opinions_in_time: List[PoliticiansOpinionInTime], politician_name: str):
    """
    Get all different trajectories of opinions for a single politician.

    Parameters:
    - opinions_in_time: List of PoliticiansOpinionInTime instances.
    - politician_name: The name of the politician.

    Returns:
    - A list of trajectories for the specified politician.
    """
    politician_trajectories = []

    # Iterate through the list of opinions_in_time
    for opinion_in_time in opinions_in_time:
        datetime_point = opinion_in_time.datetime
        # Find the politician's opinion at this datetime_point
        politician_opinion = next((opinion.opinion_score for opinion in opinion_in_time.politician_opinions
                                   if opinion.politician_name == politician_name), None)

        if politician_opinion is not None:
            # Append the datetime_point and opinion to the trajectories
            politician_trajectories.append((datetime_point, politician_opinion))

    return politician_trajectories

# Example usage:
opinions_data = [
    PoliticiansOpinionInTime([PoliticianOpinion("Politician1", 0.8), PoliticianOpinion("Politician2", 0.5)], datetime(2023, 1, 1)),
    PoliticiansOpinionInTime([PoliticianOpinion("Politician1", 0.7), PoliticianOpinion("Politician2", 0.6)], datetime(2023, 1, 2)),
    PoliticiansOpinionInTime([PoliticianOpinion("Politician1", 0.6), PoliticianOpinion("Politician2", 0.7)], datetime(2023, 1, 3)),
]

politician_name_to_check = "Politician1"
trajectories = get_politician_trajectories(opinions_data, politician_name_to_check)

print(f"Opinion trajectories for {politician_name_to_check}: {trajectories}")


Opinion trajectories for Politician1: [(datetime.datetime(2023, 1, 1, 0, 0), 0.8), (datetime.datetime(2023, 1, 2, 0, 0), 0.7), (datetime.datetime(2023, 1, 3, 0, 0), 0.6)]


In [ ]:
@dataclass
class OpinionHistogram:
    """Class for identifying a single politician opinion"""
    bins: list[int]
    opinion: list[int] 

@dataclass
class ScoreHistogram:
    """Class for identifying a single politician opinion"""
    politician_id: list[int]
    opinion: list[int]